In [26]:
# Import Core Packages
%load_ext autoreload
%autoreload 2

import math
import numpy as np

from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import seaborn as sns
import pandas as pd
import re
import sys
import os
from pathlib import Path
import platform
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# User Inputs
basepath = Path('/media/kimqi/BK/Data/PCDisruption/Raw/Strawberry/Stb_2024-10-28_Silverstone')

# Get Session Directory Information
animal = basepath.parent.name
cur_folder = Path(basepath).name
session_name = re.sub(r'_\d{4}-\d{2}-\d{2}_', '_', cur_folder)

# Get current OS and set paths dependent on working OS.
cur_system = platform.system()

if cur_system == "Linux":  # For Linux systems
    sys.path.extend(['/home/kimqi/Documents/Repositories/NeuroPy'])  # NeuroPy Path

    # Save Directory
    save_dir = Path('/home/kimqi/Documents/Analyses/Placefields')
    
elif cur_system == "Windows":
    sys.path.extend([r'E:\Code\Repositories\NeuroPy'])  # NeuroPy Path
    save_dir = Path('E:\Analyses\Placefields')  # Save Path
    save_dir = save_dir / session_name
    save_dir.mkdir(parents=True, exist_ok=True)
    plot_dir = save_dir / 'Plots'
    plot_dir.mkdir(parents=True, exist_ok=True)

In [33]:
cur_folder

'Stb_2024-10-28_Silverstone'

In [34]:
# Test NeuroPy Imports
from neuropy import core
from neuropy import io
from neuropy import plotting
from neuropy import analyses

In [44]:
# Import NeuroPy Packages
from neuropy import core
from neuropy.core import Epoch
from neuropy.core import Signal
from neuropy.core.processdata import ProcessData

from neuropy.analyses.artifact import detect_artifact_epochs

from neuropy.plotting.signals import plot_signal_w_epochs

from neuropy.io.openephysio import *
from neuropy.io import SpykingCircusIO

In [45]:
print(Signal.__init__.__code__.co_varnames)


('self', 'traces', 'sampling_rate', 't_start', 'channel_id')


In [46]:
sess = ProcessData(basepath)
print(sess.recinfo)
print(sess.eegfile)

filename: /media/kimqi/BK/Data/PCDisruption/Raw/Strawberry/Stb_2024-10-28_Silverstone/Stb_Silverstone.xml 
# channels: 35
sampling rate: 30000
lfp Srate (downsampled): 1250

duration: 3405.69 seconds 
duration: 0.95 hours 



In [48]:
%matplotlib widget

# Parameters
buffer_add = 0.25    # Add buffer before and after detected artifact epochs, None = don't add any
art_thresh = 3       # Artifact detection threshold
art_edge_cutoff = 2  # Edge cutoff value
art_merge = 6        # Merges epochs that are too close to each other

# Get signal object
signal = sess.eegfile.get_signal()
signal.

In [15]:

# Find artifact epochs. If file exists already, load that instead of creating a new file.
art_epochs_file = sess.filePrefix.with_suffix(".art_epochs.npy")
if art_epochs_file.exists():
    art_epochs = Epoch(epochs=None, file=art_epochs_file)
    print('Existing artifact epochs file loaded')
else:
    art_epochs = detect_artifact_epochs(signal, thresh=art_thresh, edge_cutoff=art_edge_cutoff,
                                        merge=art_merge)
    # Add in a buffer to prevent erroneous detetion of start/stop of artifact as SWRs.
    if buffer_add is not None:
        art_epochs.add_epoch_buffer(buffer_add)
    sess.recinfo.write_epochs(epochs=art_epochs, ext='art')
    art_epochs.save(art_epochs_file)

print(f'Total artifact time = {art_epochs.durations.sum():0.1F} seconds')
print('Automatically detected artifact epochs:')
print(art_epochs.to_dataframe())

# Plot artifact onto an example channel
chan_plot = 15 # Select a good channel
art_fig = plot_signal_w_epochs(signal, chan_plot, art_epochs)
art_fig_name = f'{session_name}_Artifacts_Channel{chan_plot}.png'
plt.savefig(plot_dir / art_fig_name, dpi=300, bbox_inches='tight')  # Save with full path

TypeError: Signal.__init__() got an unexpected keyword argument 'source_file'